In [29]:
import numpy as np

In [274]:
r_cam = 0.15
r_hum = 3
omg_cam = 4 * np.pi
omg_hum = 0.5 * np.pi

In [275]:
#time range
t_range = np.arange(0, 5 + 0.05, 0.05)
t_len = len(t_range)
#camera coordinate
x_cam, y_cam, z_cam = 10, 4, 5
#human coordinate
x_hum = y_hum = z_hum = 10
#camera trajectory
px_cam = x_cam + r_cam * np.cos(omg_cam * t_range + np.pi)
py_cam = y_cam + np.zeros(t_len)
pz_cam = z_cam + r_cam * np.sin(omg_cam * t_range + np.pi)
#human trajectory
px_hum = x_hum + r_hum *np.sin(omg_hum * t_range)
py_hum = y_hum + r_hum * np.cos(omg_hum * t_range)
pz_hum = z_hum + np.zeros(t_len)
#d trajectory
px_d = px_hum - px_cam
py_d = py_hum - py_cam
pz_d = pz_hum - py_cam
scale = 5.43#5431.2
px_d = px_d / scale
py_d = py_d / scale
pz_d = pz_d / scale

In [276]:
#evaluation
print(t_len)
print(px_cam[0:10])
print(px_d[0:10])

101
[ 9.85        9.87864745  9.95364745 10.04635255 10.12135255 10.15
 10.12135255 10.04635255  9.95364745  9.87864745]
[0.02762431 0.0656961  0.09496426 0.12043896 0.14837909 0.183803
 0.22847494 0.28013686 0.33327961 0.38115961]


In [277]:
#motion calculation
cam_motion = np.zeros((t_len-1,3))
diff_motion = np.zeros((t_len-1,3))
for i in range(t_len-1):
    m_cam_k = np.zeros((1,3))
    m_diff_k = np.zeros((1,3))
    m_cam_k[0][0] = (px_cam[i+1] - px_cam[i]) / 0.05
    m_cam_k[0][1] = (py_cam[i+1] - py_cam[i]) / 0.05
    m_cam_k[0][2] = (pz_cam[i+1] - pz_cam[i]) / 0.05
    m_diff_k[0][0] = (px_d[i+1] - px_d[i]) / 0.05
    m_diff_k[0][1] = (py_d[i+1] - py_d[i]) / 0.05
    m_diff_k[0][2] = (pz_d[i+1] - pz_d[i]) / 0.05
    cam_motion[i] = m_cam_k
    diff_motion[i] = m_diff_k
mc_mean = cam_motion.mean(axis = 0)
md_mean = diff_motion.mean(axis = 0)

In [278]:
#evaluation
print(cam_motion[0:10])
print(diff_motion[0:10])
print(mc_mean)
print(md_mean)

[[ 0.57294902  0.         -1.76335576]
 [ 1.5         0.         -1.08981379]
 [ 1.85410197  0.          0.        ]
 [ 1.5         0.          1.08981379]
 [ 0.57294902  0.          1.76335576]
 [-0.57294902  0.          1.76335576]
 [-1.5         0.          1.08981379]
 [-1.85410197  0.          0.        ]
 [-1.5         0.         -1.08981379]
 [-0.57294902  0.         -1.76335576]]
[[ 0.76143586 -0.03406261  0.        ]
 [ 0.5853632  -0.10197782  0.        ]
 [ 0.50949392 -0.16926431  0.        ]
 [ 0.55880255 -0.23550723  0.        ]
 [ 0.70847832 -0.30029816  0.        ]
 [ 0.89343887 -0.36323766  0.        ]
 [ 1.03323829 -0.42393768  0.        ]
 [ 1.06285511 -0.48202398  0.        ]
 [ 0.95759996 -0.53713844  0.        ]
 [ 0.74262854 -0.58894126  0.        ]]
[0. 0. 0.]
[ 0.11049724 -0.11049724  0.        ]


In [279]:
def cov_cal(motion1, motion2, motion1_mean, motion2_mean):
    matrix_sum = np.zeros((3,3))
    for i in range(t_len-1):
        matrix_sum += (motion1[i] - motion1_mean).T * (motion2[i] - motion2_mean)
    result = matrix_sum/(t_len-2)
    return result
    
#correlation calculation
cov_md_mc = cov_cal(diff_motion, cam_motion, md_mean, mc_mean)
cov_mc_mc = cov_cal(cam_motion, cam_motion, mc_mean, mc_mean)

In [280]:
#evaluation
print(cov_md_mc)
print(cov_mc_mc)
cov_mc_mc * cov_mc_mc

[[-0.2930416  0.         0.       ]
 [-0.2930416  0.         0.       ]
 [-0.2930416  0.         0.       ]]
[[1.73620914 0.         1.73620914]
 [1.73620914 0.         1.73620914]
 [1.73620914 0.         1.73620914]]


array([[3.01442218, 0.        , 3.01442218],
       [3.01442218, 0.        , 3.01442218],
       [3.01442218, 0.        , 3.01442218]])

In [281]:
#optimal scale S
A = cov_md_mc * cov_mc_mc
B = cov_md_mc * cov_md_mc
S = -A.sum()/B.sum()

In [282]:
#solution
print(S)
#这个结果当时间积累越长，也就是观测数据越多的时候越接近于理论值，也就是物体的真实scale

5.924787237328563
